### **Requirements:**

In [1]:
# %pip install faker sqlite pandas jinja2

### **Bibliotecas:**

In [2]:
# BIBLIOTECAS
import pandas as pd
import numpy as np
import sqlite3
import datetime
from functions.criar_base import criar_base
from faker import Faker


### **Variáveis**

**Users**
- user_id
- name
- age
- country

**Transactions**
- user_id
- transaction_id
- transaction_date
- transaction_state
- transaction_amount

## **Criando e gerando uma base de dados aleatória através da biblioteca faker:**


In [3]:
criar_base()

# as 3 variáveis abaixo são as configurações de local da biblioteca faker
# https://fakerjs.dev/guide/localization.html

faker_us = Faker('en-US')
faker_br = Faker('pt-BR')
faker_fr = Faker('fr-FR')

countries = ['USA', 'Brazil', 'France']                                               # escolhi 3 países apenas para gerar a base de dados
fakers = {'USA': faker_us, 'Brazil': faker_br, 'France': faker_fr}                    # dicionário associando as configs aos países

n_users = 500                                                                         # número de usuários


user_id = np.arange(1, n_users + 1)                                                   # user_id = 1, 2, 3...n_users
countries_choices = np.random.choice(countries, size=n_users)                         # utilizando np.random.choice() para retornar uma lista aleatória de países com 'n_users' linhas
names = [fakers[country].name() for country in countries_choices]                     # for loop para gerar os nomes de acordo com o país
ages = np.random.randint(18, 65, size=n_users)                                        # gera uma lista aleatória de idades entre 18 e 65 anos com 'n_users' linhas

# criando o dataframe de usuários
users_df = pd.DataFrame({
    'user_id': user_id,
    'name': names,
    'age': ages,
    'country': countries_choices
})

n_transactions = n_users * np.random.randint(8000, 10000)                            # determinando um número aleatório de transações, só para variar mesmo

transaction_ids = np.arange(1, n_transactions + 1)                                   # transaction_id = 1, 2, 3...n_transactions
user_id = np.random.randint(1, n_users + 1, size=n_transactions)                     # se utilizar n_users em vez de n_users+1 a função só gerá do 1 ao 499

start_date = datetime.datetime(2019, 1, 1)                                           # menor data possível
end_date = datetime.datetime.today()                                                 # data limite é data do dia atual

date_range = pd.date_range(start=start_date, end=end_date, freq='s')                 # freq = 's' gera datas no formato TIME STAMP
transaction_dates = np.random.choice(date_range, size=n_transactions)                # (https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases)

transaction_states = ['INITIATED', 'SUCCESS', 'FRAUD', 'CANCELLED']                  # todos os possíveis valores para 'transaction_states'
transaction_states_list = np.random.choice(transaction_states, size=n_transactions)  # gera uma lista aleatória com 'n_transactions' itens de possíveis estados
transaction_amounts = np.round(np.random.uniform(500, 5000, size=n_transactions), 2) # gera uma lista aleatória com 'n_transactions' itens de valores aleatórios

# criando o dataframe de transações
transactions_df = pd.DataFrame({
    'transaction_id': transaction_ids,
    'user_id': user_id,
    'transaction_date': transaction_dates,
    'transaction_state': transaction_states_list,
    'transaction_amount': transaction_amounts
})

transactions_df['transaction_date'] = transactions_df['transaction_date'].astype(str)

display(users_df)
display(transactions_df)


,user_id,name,age,country
0,1,Antoine Guillon,31,France
1,2,Lucy du Moulin,29,France
2,3,Roger Collet,54,France
3,4,Maria Sophia Ramos,23,Brazil
4,5,Martin Johnston,45,USA
...,...,...,...,...
495,496,Raquel da Cruz,61,Brazil
496,497,Gary Roth,50,USA
497,498,Isaac Williams,22,USA
498,499,Kimberly Braun,21,USA


,transaction_id,user_id,transaction_date,transaction_state,transaction_amount
0,1,385,2019-03-27 14:29:52,FRAUD,3021.95
1,2,350,2022-07-09 01:10:11,INITIATED,4607.62
2,3,222,2020-08-16 14:09:15,CANCELLED,2036.47
3,4,126,2024-12-18 08:46:42,SUCCESS,2255.30
4,5,274,2024-05-11 10:36:16,CANCELLED,1033.71
...,...,...,...,...,...
4125495,4125496,170,2024-03-02 22:55:06,CANCELLED,1839.84
4125496,4125497,493,2023-10-10 10:07:38,INITIATED,3863.97
4125497,4125498,85,2019-10-18 05:51:27,INITIATED,4734.53
4125498,4125499,242,2023-11-22 14:34:08,FRAUD,2797.85


## **Criando as tabelas users e transactions**

In [4]:
# conecta ao banco do SQLite
conn = sqlite3.connect('dbCompany')
cursor = conn.cursor()

users_data = list(users_df.itertuples(index=False, name=None))

transactions_data = list(transactions_df.itertuples(index=False, name=None))

# inserindo users_df na tabela 'users'
cursor.executemany('INSERT INTO users (user_id, name, age, country) VALUES (?, ?, ?, ?)', users_data)

# inserindo transactions_df na tabela 'transactions'
cursor.executemany(
                    '''
                    INSERT INTO transactions 
                    (transaction_id, user_id, transaction_date, transaction_state, transaction_amount) 
                    VALUES (?, ?, ?, ?, ?)
                    '''
                    , transactions_data
)

conn.commit()

print(f'Número de usuários inseridos: {len(users_data):,}\nNúmero de transações inseridas: {len(transactions_data):,}')

Número de usuários inseridos: 500
Número de transações inseridas: 4,125,500


## **Questões:**
**A.** Qual é a idade média de usuários do sistema por país

In [5]:
## QUERY A
query = '''
        SELECT country AS País, ROUND(AVG(age),2) AS "Média de Idade"
        FROM users
        GROUP BY country
        ORDER BY "Média de Idade" DESC
        '''

df_A = pd.read_sql(query, conn)

**B.** Qual é o país com a maior quantidade de dinheiro transacionado (considere só transações finalizadas com sucesso ou ``SUCCESS``)

In [6]:
## QUERY B
query = '''
        WITH TotalSuccess AS (
                SELECT user_id, SUM(transaction_amount) AS transacoes
                FROM transactions 
                WHERE transaction_state = 'SUCCESS'
                GROUP BY user_id
                )

        SELECT u.country AS País, 
               SUM(t.transacoes) AS "Soma das Transações (US$)"
        FROM TotalSuccess t
        LEFT JOIN users u
        ON u.user_id = t.user_id
        GROUP BY u.country
        ORDER BY "Soma das Transações (US$)" DESC
        '''

df_B = pd.read_sql(query, conn)

## formatações no dataframe para melhor exibir as respostas no final
df_B['Soma das Transações (US$)'] = df_B['Soma das Transações (US$)'].apply(lambda x: f"{x:,.2f}")

**C.** Qual é o país com maior taxa de fraude em porcentagem respeito ao número de transações totais no país

In [7]:
## QUERY C
query = '''
        WITH
        Total AS (
            SELECT u.country, COUNT(*) as total_transacoes
            FROM transactions t
            LEFT JOIN users u
            ON t.user_id = u.user_id 
            GROUP BY u.country
        ),
        Fraudes AS (
            SELECT u.country, COUNT(*) as total_fraud
            FROM transactions t
            LEFT JOIN users u
            ON t.user_id = u.user_id
            WHERE t.transaction_state = 'FRAUD'
            GROUP BY u.country
        )
        
        SELECT 
            t.country AS País,
            ROUND(CAST(f.total_fraud AS FLOAT) / t.total_transacoes * 100, 2) as "Taxa de Fraudes (%)"
        FROM Total t
        LEFT JOIN Fraudes f
        ON t.country = f.country
        ORDER BY "Taxa de Fraudes (%)" DESC
        '''


df_C = pd.read_sql(query, conn)

**D.** Na mesma linha da pergunta anterior, responda qual é a faixa de idade de usuários
que mais cometem fraude (em percentagem).

Separe as faixas etárias em ``< 18 anos, 18-30 anos, 30 - 45 anos, 45 - 60 anos, 60 > anos``

Considerar o fato que um usuário pode ter executado várias transações, das quais poucas (ou muitas) podem ter sido fraude entre as demais.

In [8]:
## QUERY D
query = '''      
        SELECT 
            CASE 
                WHEN age < 18 THEN '< 18 anos'
                WHEN age BETWEEN 18 AND 30 THEN '18-30 anos'
                WHEN age BETWEEN 30 AND 45 THEN '30 - 45 anos'
                WHEN age BETWEEN 45 AND 60 THEN '45 - 60 anos'
                ELSE '60 > anos'
            END AS "Faixa etária",
            COUNT(*) AS "Qtd. Transações Fraudulentas",
            ROUND(CAST(COUNT(*) AS FLOAT) / (SELECT COUNT(*) FROM transactions WHERE transaction_state = 'FRAUD') * 100, 2) AS "Taxa de Fraudes (%)"
        FROM users u
        JOIN transactions t ON u.user_id = t.user_id
        
        WHERE t.transaction_state = 'FRAUD'
        GROUP BY "Faixa etária"
        ORDER BY "Faixa etária" ASC
        '''

df_D = pd.read_sql(query, conn)

## formatações no dataframe para melhor exibir as respostas no final
df_D["Qtd. Transações Fraudulentas"] = df_D["Qtd. Transações Fraudulentas"].apply(lambda x: f"{x:,.0f}")

**E.** Imagine que a camada executiva da empresa dona do sistema, precisa criar um Dashboard para monitorar o estado das transações nos últimos 3 dias.

Criar uma query SQL que calcule:

1. ``Número e dinheiro das transações não finalizadas``

2. ``Número e dinheiro de transações finalizadas com sucesso (SUCCESS)``

3. ``Número e dinheiro de transações canceladas (CANCELLED)``

4. ``Número e dinheiro de fraudes (FRAUD)``

Agrupado por país e nos 3 dias anteriores de quando o executivo da empresa consulte seu Dashboard.

In [9]:
## QUERY E:
query = '''      
        WITH TabelaAux AS (
            SELECT 
                u.country,
                t.transaction_state,
                COUNT(*) AS transaction_count,
                SUM(t.transaction_amount) AS transaction_amount
            FROM transactions t
            JOIN users u ON u.user_id = t.user_id

            WHERE t.transaction_date <= DATE('now', '-3 day')
                -- SQLite não possui as funções CURDATE() e INTERVAL
                -- WHERE t.transaction_date <= CURDATE() - INTERVAL 3 DAY)

            GROUP BY u.country, t.transaction_state
        )
        SELECT 
            country as País,
            SUM(CASE WHEN transaction_state = 'INITIATED' THEN transaction_count ELSE 0 END) AS "N. Não Finalizadas",
            SUM(CASE WHEN transaction_state = 'INITIATED' THEN transaction_amount ELSE 0 END) AS "Soma Não Finalizadas",
            
            SUM(CASE WHEN transaction_state = 'SUCCESS' THEN transaction_count ELSE 0 END) AS "N. Finalizadas",
            SUM(CASE WHEN transaction_state = 'SUCCESS' THEN transaction_amount ELSE 0 END) AS "Soma Finalizadas",
            
            SUM(CASE WHEN transaction_state = 'CANCELLED' THEN transaction_count ELSE 0 END) AS "N. Canceladas",
            SUM(CASE WHEN transaction_state = 'CANCELLED' THEN transaction_amount ELSE 0 END) AS "Soma Canceladas",
            
            SUM(CASE WHEN transaction_state = 'FRAUD' THEN transaction_count ELSE 0 END) AS "N. Fraudulentas",
            SUM(CASE WHEN transaction_state = 'FRAUD' THEN transaction_amount ELSE 0 END) AS "Soma Fraudulentas"
        FROM TabelaAux
        GROUP BY country
        '''

df_E = pd.read_sql(query, conn)

## formatações no dataframe para melhor exibir as respostas no final
df_E["N. Não Finalizadas"] = df_E["N. Não Finalizadas"].apply(lambda x: f"{x:,.0f}")
df_E["N. Finalizadas"] = df_E["N. Finalizadas"].apply(lambda x: f"{x:,.0f}")
df_E["N. Canceladas"] = df_E["N. Canceladas"].apply(lambda x: f"{x:,.0f}")
df_E["N. Fraudulentas"] = df_E["N. Fraudulentas"].apply(lambda x: f"{x:,.0f}")

df_E["Soma Não Finalizadas"] = df_E["Soma Não Finalizadas"].apply(lambda x: f"{x:,.2f}")
df_E["Soma Finalizadas"] = df_E["Soma Finalizadas"].apply(lambda x: f"{x:,.2f}")
df_E["Soma Canceladas"] = df_E["Soma Canceladas"].apply(lambda x: f"{x:,.2f}")
df_E["Soma Fraudulentas"] = df_E["Soma Fraudulentas"].apply(lambda x: f"{x:,.2f}")


### **Respostas:**

In [10]:
# IMPRIMIR OS RESULTADOS
print('Questão A:')
display(df_A)
print('Questão B:')
display(df_B)
print('Questão C:')
display(df_C)
print('Questão D:')
display(df_D)
print('Questão E:')
display(df_E)
# conn.close()

Questão A:


,País,Média de Idade
0,Brazil,41.58
1,USA,41.21
2,France,40.63


Questão B:


,País,Soma das Transações (US$)
0,France,"1,039,618,353.27"
1,Brazil,"970,610,167.45"
2,USA,"829,749,420.07"


Questão C:


,País,Taxa de Fraudes (%)
0,Brazil,24.98
1,France,24.91
2,USA,24.90


Questão D:


,Faixa etária,Qtd. Transações Fraudulentas,Taxa de Fraudes (%)
0,18-30 anos,"269,011",26.15
1,30 - 45 anos,"347,443",33.78
2,45 - 60 anos,"333,872",32.46
3,60 > anos,"78,203",7.60


Questão E:


,País,N. Não Finalizadas,Soma Não Finalizadas,N. Finalizadas,Soma Finalizadas,N. Canceladas,Soma Canceladas,N. Fraudulentas,Soma Fraudulentas
0,Brazil,"352,153","968,395,830.73","352,633","969,388,785.41","352,428","968,455,137.34","351,973","968,267,963.48"
1,France,"376,710","1,036,099,318.50","377,895","1,038,256,254.37","377,088","1,038,114,370.11","375,538","1,034,400,440.55"
2,USA,"301,375","828,346,224.04","301,135","828,770,045.65","301,676","829,015,821.13","299,753","825,275,819.02"
